In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import noisereduce as nr
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

C:\Users\Nithya Thesis\anaconda3\lib\site-packages\noisereduce\noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
header = 'filename spectral_centroid rmse rolloff zero_crossing_rate'
for i in range(1, 14):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [5]:
file = open('audio_class.csv', 'w', newline='')

with file:
    writer = csv.writer(file)
    writer.writerow(header)
    
    
audio_clips = 'clap cough music talking train'.split()

for a in audio_clips:
    for filename in os.listdir(f'dataa/{a}'):
        audio = f'dataa/{a}/{filename}'
        y, sr = librosa.load(audio, mono=True, sr=22050)
        
        # noise reduction
        noisy_part = y[0:10000]  
        reduced_noise = nr.reduce_noise(audio_clip=y, noise_clip=noisy_part, verbose=False)
    
        #trimming
        trimmed, index = librosa.effects.trim(reduced_noise, top_db=20, frame_length=512, hop_length=64)
        
        #features
        
        spec_cent = librosa.feature.spectral_centroid(y=trimmed, sr=sr)
        
        rmse = librosa.feature.rms(y=trimmed)
        
        rolloff = librosa.feature.spectral_rolloff(y=trimmed, sr=sr)
        
        zcr = librosa.feature.zero_crossing_rate(trimmed)
        
        mfcc = librosa.feature.mfcc(y=trimmed, sr=sr,n_mfcc=13)
        
        to_append = f'{filename} {np.mean(spec_cent)} {np.mean(rmse)} {np.mean(rolloff)} {np.mean(zcr)} '    
        
        for m in mfcc:
            to_append += f' {np.mean(m)}'
            
        to_append += f' {a}'
        
        file = open('audio_class.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [33]:
data = pd.read_csv('audio_class.csv')
data.head()# Dropping unneccesary columns


,filename,spectral_centroid,rmse,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,label
clap,(1).wav,2051.572081,0.008992,4060.966353,0.134100,-605.536610,56.630991,-23.340395,11.531782,-11.342088,7.796704,-12.405108,2.648839,0.432009,1.499365,-1.880858,-1.527689,0.625778,clap
clap,(10).wav,2058.214164,0.005593,4190.654963,0.113909,-658.675436,32.362119,-18.051734,10.146358,-5.399427,10.915433,-5.938106,4.143925,-2.083303,-0.279218,-1.515553,-3.099555,-1.432691,clap
clap,(100).wav,2843.481250,0.002820,5076.793352,0.136552,-686.391418,19.337809,11.524174,51.081119,-2.689775,14.175341,22.585746,16.809678,-5.130494,12.618686,25.561120,1.495639,-1.316608,clap
clap,(101).wav,2224.800218,0.001412,4658.933844,0.149073,-610.523518,101.509821,-81.046570,-23.512760,-66.012875,24.605814,-56.382617,-13.351940,-30.259235,-5.224484,-16.138139,-30.078784,-4.236884,clap
clap,(102).wav,1275.823458,0.004507,1788.112294,0.084689,-518.568838,165.982720,-46.998002,-47.874393,-24.953285,-3.947731,-17.928818,-21.016032,-14.468609,-2.527688,-7.698056,-14.593245,-6.349583,clap
